For our analysis, we use the data found in multi asset xlsx.
The time-series data gives monthly returns for the 11 asset classes from mid-2009 to mid-2021.
We will be working with the risky MV frontier for 11 risky asset classes, and we will use the excess-return formulation and frontier. The data you are provided has already subtracted short- term treasury returns, (a measure of the risk-free rate.) So go ahead and use these returns as excess returns on risky
 

In [19]:
import pandas as pd
import numpy as np
import sklearn 
import PyPortfolioOpt
import datetime

In [7]:
df = pd.read_csv("multi_asset_returns.csv")
df.head()

,Date,SPY,EFA,EEM,PSP,QAI,HYG,DBC,IYR,IEF,BWX,TIP
0,YYYY-00-DD 00:00:SS,0.098792,0.114636,0.155028,0.229649,0.022328,0.137877,-0.001554,0.295597,-0.028010,0.008439,-0.018522
1,YYYY-00-DD 00:00:SS,0.058925,0.132389,0.159871,0.054364,0.028336,0.028966,0.163134,0.023199,-0.020289,0.054141,0.020495
2,YYYY-00-DD 00:00:SS,-0.001274,-0.014910,-0.023135,0.044844,-0.004035,0.032761,-0.026857,-0.025696,-0.006119,0.004553,0.001410
3,YYYY-00-DD 00:00:SS,0.074632,0.100441,0.110172,0.143273,0.015352,0.069189,0.018594,0.105825,0.008339,0.031311,0.000910
4,YYYY-00-DD 00:00:SS,0.036502,0.044593,-0.013573,0.032975,-0.004589,-0.017428,-0.040802,0.131501,0.007227,0.007191,0.007999


Summary Statistics
 

Calculate and display the mean and volatility of each asset’s excess (Recall we use volatility to refer to standard deviation.)
Which assets have the best and worst Sharpe ratios?

In [27]:
def sharpe_ratio(column,rf=0):
    return (column.mean() - rf)/ np.std(column)

In [31]:
for i in df.columns:
    if i == 'Date':
        continue
    else:
        print("{0}: mu is {1} std is {2} Sharpe Ratio is {3}".format(i,df[i].mean(),df[i].std(),sharpe_ratio(df[i])))

SPY: mu is 0.01347640676791893 std is 0.04040836552779444 Sharpe Ratio is 0.33464555943368657
EFA: mu is 0.008320893355705105 std is 0.046877398139852686 Sharpe Ratio is 0.17811016115765788
EEM: mu is 0.008240717355153242 std is 0.056731502701604895 Sharpe Ratio is 0.14575483525435834
PSP: mu is 0.01314823816239015 std is 0.062329498862949285 Sharpe Ratio is 0.21166846799309053
QAI: mu is 0.0022598877819213193 std is 0.014055689757563529 Sharpe Ratio is 0.16133067502530488
HYG: mu is 0.006805487315571347 std is 0.024915598645360047 Sharpe Ratio is 0.274075453852876
DBC: mu is 0.0007352086989903408 std is 0.05145793151885108 Sharpe Ratio is 0.014336414592683693
IYR: mu is 0.013602713863471777 std is 0.05231443453882066 Sharpe Ratio is 0.2609073217385856
IEF: mu is 0.0026348746030401093 std is 0.01648108126302998 Sharpe Ratio is 0.16041925316813094
BWX: mu is 0.0017477300240697237 std is 0.021536344380396137 Sharpe Ratio is 0.0814300308309291
TIP: mu is 0.0029218380542438928 std is 0.012

Best Sharpe Ratio is SPY and worst Sharpe Ratio is DBC

The MV frontier
Compute and display the weights of the tangency portfolios: wtan.
Compute the mean, volatility, and Sharpe ratio for the tangency portfolio corresponding to wtan.

In [47]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [54]:
df

,Date,SPY,EFA,EEM,PSP,QAI,HYG,DBC,IYR,IEF,BWX,TIP
0,YYYY-00-DD 00:00:SS,0.098792,0.114636,0.155028,0.229649,0.022328,0.137877,-0.001554,0.295597,-0.028010,0.008439,-0.018522
1,YYYY-00-DD 00:00:SS,0.058925,0.132389,0.159871,0.054364,0.028336,0.028966,0.163134,0.023199,-0.020289,0.054141,0.020495
2,YYYY-00-DD 00:00:SS,-0.001274,-0.014910,-0.023135,0.044844,-0.004035,0.032761,-0.026857,-0.025696,-0.006119,0.004553,0.001410
3,YYYY-00-DD 00:00:SS,0.074632,0.100441,0.110172,0.143273,0.015352,0.069189,0.018594,0.105825,0.008339,0.031311,0.000910
4,YYYY-00-DD 00:00:SS,0.036502,0.044593,-0.013573,0.032975,-0.004589,-0.017428,-0.040802,0.131501,0.007227,0.007191,0.007999
...,...,...,...,...,...,...,...,...,...,...,...,...
142,YYYY-00-DD 00:00:SS,0.027806,0.022379,0.007878,0.045824,0.001877,-0.002455,0.101382,0.024273,-0.023638,-0.026853,-0.016876
143,YYYY-00-DD 00:00:SS,0.045496,0.025222,-0.007168,0.031376,-0.007090,0.012204,-0.007082,0.057787,-0.023767,-0.027598,-0.002532
144,YYYY-00-DD 00:00:SS,0.052910,0.029524,0.011999,0.076870,0.005975,0.006364,0.078266,0.078856,0.010007,0.017251,0.014291
145,YYYY-00-DD 00:00:SS,0.006566,0.034823,0.016488,0.018566,0.005939,0.000388,0.038526,0.010082,0.004252,0.009824,0.010231


In [55]:
df = df.drop("Date",axis=1)

In [56]:
# Calculate expected returns and sample covariance
mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)

# Optimize for maximal Sharpe ratio
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
ef.portfolio_performance(verbose=True)

ERROR in LDL_factor: Error in KKT matrix LDL factorization when computing the nonzero elements. The problem seems to be non-convex
ERROR in osqp_setup: KKT matrix factorization.
The problem seems to be non-convex.


ValueError: Workspace allocation error!

In [44]:
r = df.drop("Date",axis=1) #r will be the row's of this
w = np.array([1/len(r.columns)]*len(r.columns)) #start with an equally weighted portfolio
mus = []
for i in range(len(r)):
    mus.append(w * r.loc[i])

In [45]:
mus

[SPY    0.008981
 EFA    0.010421
 EEM    0.014093
 PSP    0.020877
 QAI    0.002030
 HYG    0.012534
 DBC   -0.000141
 IYR    0.026872
 IEF   -0.002546
 BWX    0.000767
 TIP   -0.001684
 Name: 0, dtype: float64,
 SPY    0.005357
 EFA    0.012035
 EEM    0.014534
 PSP    0.004942
 QAI    0.002576
 HYG    0.002633
 DBC    0.014830
 IYR    0.002109
 IEF   -0.001844
 BWX    0.004922
 TIP    0.001863
 Name: 1, dtype: float64,
 SPY   -0.000116
 EFA   -0.001355
 EEM   -0.002103
 PSP    0.004077
 QAI   -0.000367
 HYG    0.002978
 DBC   -0.002442
 IYR   -0.002336
 IEF   -0.000556
 BWX    0.000414
 TIP    0.000128
 Name: 2, dtype: float64,
 SPY    0.006785
 EFA    0.009131
 EEM    0.010016
 PSP    0.013025
 QAI    0.001396
 HYG    0.006290
 DBC    0.001690
 IYR    0.009620
 IEF    0.000758
 BWX    0.002846
 TIP    0.000083
 Name: 3, dtype: float64,
 SPY    0.003318
 EFA    0.004054
 EEM   -0.001234
 PSP    0.002998
 QAI   -0.000417
 HYG   -0.001584
 DBC   -0.003709
 IYR    0.011955
 IEF    0.00

The allocation
Compute and display the weights of MV portfolios with target returns of µp = 0.0075 (
This is monthly data, so while this target looks small, it is reasonable.)

What is the mean, volatility, and Sharpe ratio for wp?
Discuss the In which assets is the portfolio most long? And short?
Does this line up with which assets have the strongest Sharpe ratios?

In [58]:
# get weights maximizing the Sharpe ratio 
ef = EfficientFrontier(avg_returns, cov_mat) 
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights() 
cleaned_weights

NameError: name 'avg_returns' is not defined

Long-short positions
 

Consider an allocation between only domestic and foreign (Drop all other return columns and recompute wp for µp = .0075.)
Is the portfolio balanced?
Make an adjustment to µforeign equities of +0.002. Recompute wp for µp = .0075 for these two assets
How does the allocation among the two assets change?

Does this two-asset example raise any issues for the 11-asset problem about the statistical precision of the MV solutions?

Robustness
Recalculate the full allocation, again with the unadjusted µ foreign equities and again for µp = 0. This time, make one change: in building wtan, do not use Σ as given in the formulas in the lecture. Rather, use a diaganolized ΣD, which zeroes out all non-diagonal elements of the full covariance matrix, Σ.
How does the allocation look now?

What does this suggest about the sensitivity of the solution to estimated means and esti- mated covariances?
HMC deals with this sensitivity by using explicit constraints on the allocation Con- ceptually, what are the pros/cons of doing that versus modifying the formula with ΣD?
Out-of-Sample Performance
Let’s divide the sample to both compute a portfolio  and  then  check  its  performance  out  of sample.

Using only data through the end of 2018, compute wp for µp = .0075, allocating to all 11
Calculate the portfolio’s Sharpe ratio within that sample, through the end of
Calculate the portfolio’s Sharpe ratio based on performance in 2019-2021.
How does this out-of-sample Sharpe compare to the 2009-2018 performance of a portfolio optimized to µp using 2009-2018 data?
Robust Out-of-Sample Performance
Recalculate wp on 2009-2018 data using the diaganolized covariance matrix, ΣD. What is the performance of this portfolio in 2019-2021? Does it do better out of of sample than the portfolio constructed on 2009-2018 data using the full covariance matrix?